# Packages

In [ ]:
%%capture
! pip install deeppavlov

In [ ]:
%%capture
!pip install tensorflow-gpu==1.15.2


In [ ]:
%%capture
! python -m deeppavlov install morpho_ru_syntagrus_bert
! pip install pyconll


In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data and model

## Data

Download data and save it to data folder

In [ ]:
%%capture
! wget https://raw.githubusercontent.com/dialogue-evaluation/GramEval2020/master/dataTrain/GramEval2020-Taiga-social-train.conllu
! wget https://raw.githubusercontent.com/dialogue-evaluation/GramEval2020/master/dataTrain/MorphoRuEval2017-VK-gold.conllu
! wget https://raw.githubusercontent.com/dialogue-evaluation/GramEval2020/master/dataOpenTest/GramEval2020-RuEval2017-social-dev.conllu

In [ ]:
import os
os.makedirs('/content/data/GramEval/social')

os.rename("/content/GramEval2020-RuEval2017-social-dev.conllu", "/content/data/GramEval/social/ru_social-ud-dev.conllu")

train1 = open('/content/GramEval2020-Taiga-social-train.conllu','r') 
train2 = open('/content/MorphoRuEval2017-VK-gold.conllu','r')
train_all = open('/content/data/GramEval/social/ru_social-ud-train.conllu', 'w')

train_all.write(train1.read())
train_all.write(train2.read())

train1.close()
train2.close()
train_all.close()

In [ ]:
datafolder_path = '/content/data/GramEval/social' #path to datafolder
models_path = '/content/drive/MyDrive/models_diploma/GramEval_social'
language_conf = 'ru_social'

## RuBERT

download bert model, don't know why though

In [ ]:
%%capture
!wget http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz

In [ ]:
import tarfile

file = tarfile.open('./rubert_cased_L-12_H-768_A-12_v1.tar.gz')
  
# extracting file
file.extractall('./downloads/bert_models')
  
file.close()

# Count and split tokens by frequency

In [ ]:
from utils import divide_by_freq

top100, top1000, other = divide_by_freq(datafolder_path+'/ru_syntagrus-ud-train.conllu')

os.makedirs(datafolder_path + '/freq_groups')

for i, x in enumerate([top100, top1000, other]):
    file_path = datafolder_path + '/freq_groups' + '/{}.txt'.format(i)
    with open(file_path, 'w') as f:
        f.write(','.join(x))


# Components

## Preprocessor

In [ ]:
import re
import random
from logging import getLogger
from typing import Tuple, List, Optional, Union

from deeppavlov.core.common.registry import register
from bert_dp.preprocessing import convert_examples_to_features, InputExample, InputFeatures
from bert_dp.tokenization import FullTokenizer

from deeppavlov.core.commands.utils import expand_path
from deeppavlov.core.data.utils import zero_pad
from deeppavlov.core.models.component import Component
from deeppavlov.models.preprocessors.mask import Mask


@register('my_bert_ner_preprocessor')
class MyBertNerPreprocessor(Component):
    """Takes tokens and splits them into bert subtokens, encodes subtokens with their indices.
    Creates a mask of subtokens (one for the first subtoken, zero for the others).
    If tags are provided, calculates tags for subtokens.
    Args:
        vocab_file: path to vocabulary
        do_lower_case: set True if lowercasing is needed
        max_seq_length: max sequence length in subtokens, including [SEP] and [CLS] tokens
        max_subword_length: replace token to <unk> if it's length is larger than this
            (defaults to None, which is equal to +infinity)
        token_masking_prob: probability of masking token while training
        provide_subword_tags: output tags for subwords or for words
        subword_mask_mode: subword to select inside word tokens, can be "first" or "last"
            (default="first")
    Attributes:
        max_seq_length: max sequence length in subtokens, including [SEP] and [CLS] tokens
        max_subword_length: rmax lenght of a bert subtoken
        tokenizer: instance of Bert FullTokenizer

    !!Added: 
        topk_tokens: path to file with tokens to consider in topk_tokens_mask in train mode
        if list of files is passed,  condiser all the tokens apart from the ones included in the files
    """

    def __init__(self,
                 vocab_file: str,
                 topk_tokens_path: List[str] = None, # added
                 last: bool = False,
                 do_lower_case: bool = False,
                 max_seq_length: int = 512,
                 max_subword_length: int = None,
                 token_masking_prob: float = 0.0,
                 provide_subword_tags: bool = False,
                 subword_mask_mode: str = "first",
                 **kwargs):
        self._re_tokenizer = re.compile(r"[\w']+|[^\w ]")
        self.provide_subword_tags = provide_subword_tags
        self.mode = kwargs.get('mode')
        self.max_seq_length = max_seq_length
        self.max_subword_length = max_subword_length
        self.subword_mask_mode = subword_mask_mode
        vocab_file = str(expand_path(vocab_file))
        self.tokenizer = FullTokenizer(vocab_file=vocab_file,
                                       do_lower_case=do_lower_case)
        self.token_masking_prob = token_masking_prob

        self.last = None # added
        self.topk_tokens = [] # added
        for filename in topk_tokens_path: # added
            with open(filename, 'r') as f: # added
                  tokens = f.read().split(',') # added
            self.topk_tokens.extend(tokens) # added
        
       
        self.last = last
        

    def __call__(self,
                 tokens: Union[List[List[str]], List[str]],
                 tags: List[List[str]] = None,
                 **kwargs):
      
        if isinstance(tokens[0], str):
            tokens = [re.findall(self._re_tokenizer, s) for s in tokens]
        subword_tokens, subword_tok_ids, startofword_markers, subword_tags, topk_tok_mask = [], [], [], [], []
        for i in range(len(tokens)):
            toks = tokens[i]
            ys = ['O'] * len(toks) if tags is None else tags[i]
            assert len(toks) == len(ys), \
                f"toks({len(toks)}) should have the same length as ys({len(ys)})"
            sw_toks, sw_marker, sw_ys, tk_masks = \
                self._ner_bert_tokenize(toks,
                                        ys,
                                        self.topk_tokens, # added
                                        self.tokenizer,
                                        self.max_subword_length,
                                        mode=self.mode,
                                        last=self.last,
                                        subword_mask_mode=self.subword_mask_mode,
                                        token_masking_prob=self.token_masking_prob)
            if self.max_seq_length is not None:
                if len(sw_toks) > self.max_seq_length:
                    raise RuntimeError(f"input sequence after bert tokenization"
                                       f" shouldn't exceed {self.max_seq_length} tokens.")
            topk_tok_mask.append(tk_masks) # added
            subword_tokens.append(sw_toks)
            subword_tok_ids.append(self.tokenizer.convert_tokens_to_ids(sw_toks))
            startofword_markers.append(sw_marker)
            subword_tags.append(sw_ys)
            assert len(sw_marker) == len(sw_toks) == len(subword_tok_ids[-1]) == len(sw_ys), \
                f"length of sow_marker({len(sw_marker)}), tokens({len(sw_toks)})," \
                f" token ids({len(subword_tok_ids[-1])}) and ys({len(ys)})" \
                f" for tokens = `{toks}` should match"

        subword_tok_ids = zero_pad(subword_tok_ids, dtype=int, padding=0)
        startofword_markers = zero_pad(startofword_markers, dtype=int, padding=0)
        attention_mask = Mask()(subword_tokens)
        topk_tok_mask = zero_pad(topk_tok_mask, dtype=int, padding=0) # added


        if tags is not None:
            if self.provide_subword_tags:
                return tokens, subword_tokens, subword_tok_ids, \
                    attention_mask, startofword_markers, subword_tags
            else:
                nonmasked_tags = [[t for t in ts if t != 'X'] for ts in tags]
                for swts, swids, swms, ts in zip(subword_tokens,
                                                 subword_tok_ids,
                                                 startofword_markers,
                                                 nonmasked_tags):
                    if (len(swids) != len(swms)) or (len(ts) != sum(swms)):
                        log.warning('Not matching lengths of the tokenization!')
                        log.warning(f'Tokens len: {len(swts)}\n Tokens: {swts}')
                        log.warning(f'Markers len: {len(swms)}, sum: {sum(swms)}')
                        log.warning(f'Masks: {swms}')
                        log.warning(f'Tags len: {len(ts)}\n Tags: {ts}')
                return tokens, subword_tokens, subword_tok_ids, \
                    attention_mask, startofword_markers, nonmasked_tags

        return tokens, subword_tokens, subword_tok_ids, startofword_markers, attention_mask, topk_tok_mask

    @staticmethod
    def _ner_bert_tokenize(tokens: List[str],
                           tags: List[str],
                           topk_tokens: List[str],
                           tokenizer: FullTokenizer,
                           max_subword_len: int = None,
                           mode: str = None,
                           last: str = None,
                           subword_mask_mode: str = "first",
                           token_masking_prob: float = None) -> Tuple[List[str], List[int], List[str]]:
        do_masking = (mode == 'train') and (token_masking_prob is not None)
        do_cutting = (max_subword_len is not None)
        tokens_subword = ['[CLS]']
        topk_tokens_mask = [] # added
        startofword_markers = [0]
        tags_subword = ['X']
        for token, tag in zip(tokens, tags):
            token_marker = int(tag != 'X')
            subwords = tokenizer.tokenize(token)
            if not subwords or (do_cutting and (len(subwords) > max_subword_len)):
                tokens_subword.append('[UNK]')
                startofword_markers.append(token_marker)
                tags_subword.append(tag)
            else:
                if do_masking and (random.random() < token_masking_prob):
                    tokens_subword.extend(['[MASK]'] * len(subwords))
                else:
                    tokens_subword.extend(subwords)

                if subword_mask_mode == "last":
                    startofword_markers.extend([0] * (len(subwords) - 1) + [token_marker])
                else: #subword_mask_mode=first
                    startofword_markers.extend([token_marker] + [0] * (len(subwords) - 1))

                if token.lower() in topk_tokens:  # added
                    if last == True:  # added
                        topk_tokens_mask.extend([0]) # added
                    else:
                        topk_tokens_mask.extend([1]) # added
                else:
                    if last == True:   # added
                        topk_tokens_mask.extend([1]) # added
                    else:
                        topk_tokens_mask.extend([0]) # added


                tags_subword.extend([tag] + ['X'] * (len(subwords) - 1))
                
        tokens_subword.append('[SEP]')
        # topk_tokens_mask.append(0)
        startofword_markers.append(0)
        tags_subword.append('X')
        return tokens_subword, startofword_markers, tags_subword, topk_tokens_mask


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.


## Tagger


In [ ]:
from logging import getLogger
from typing import List, Union, Dict, Optional

import numpy as np
import tensorflow as tf
from bert_dp.modeling import BertConfig, BertModel
from bert_dp.optimization import AdamWeightDecayOptimizer

from deeppavlov.core.commands.utils import expand_path
from deeppavlov.core.common.registry import register
from deeppavlov.core.layers.tf_layers import bi_rnn
from deeppavlov.core.models.tf_model import LRScheduledTFModel

log = getLogger(__name__)


def token_from_subtoken(units: tf.Tensor, mask: tf.Tensor) -> tf.Tensor:
    """ Assemble token level units from subtoken level units
    Args:
        units: tf.Tensor of shape [batch_size, SUBTOKEN_seq_length, n_features]
        mask: mask of token beginnings. For example: for tokens
                [[``[CLS]`` ``My``, ``capybara``, ``[SEP]``],
                [``[CLS]`` ``Your``, ``aar``, ``##dvark``, ``is``, ``awesome``, ``[SEP]``]]
            the mask will be
                [[0, 1, 1, 0, 0, 0, 0],
                [0, 1, 1, 0, 1, 1, 0]]
    Returns:
        word_level_units: Units assembled from ones in the mask. For the
            example above this units will correspond to the following
                [[``My``, ``capybara``],
                [``Your`, ``aar``, ``is``, ``awesome``,]]
            the shape of this tensor will be [batch_size, TOKEN_seq_length, n_features]
    """
    shape = tf.cast(tf.shape(units), tf.int64)
    batch_size = shape[0]
    nf = shape[2]
    nf_int = units.get_shape().as_list()[-1]

    # number of TOKENS in each sentence
    token_seq_lengths = tf.cast(tf.reduce_sum(mask, 1), tf.int64)
    # for a matrix m =
    # [[1, 1, 1],
    #  [0, 1, 1],
    #  [1, 0, 0]]
    # it will be
    # [3, 2, 1]

    n_words = tf.reduce_sum(token_seq_lengths)
    # n_words -> 6

    max_token_seq_len = tf.cast(tf.reduce_max(token_seq_lengths), tf.int64)
    # max_token_seq_len -> 3

    idxs = tf.where(mask)
    # for the matrix mentioned above
    # tf.where(mask) ->
    # [[0, 0],
    #  [0, 1]
    #  [0, 2],
    #  [1, 1],
    #  [1, 2]
    #  [2, 0]]

    sample_ids_in_batch = tf.pad(idxs[:, 0], [[1, 0]])
    # for indices
    # [[0, 0],
    #  [0, 1]
    #  [0, 2],
    #  [1, 1],
    #  [1, 2],
    #  [2, 0]]
    # it is
    # [0, 0, 0, 0, 1, 1, 2]
    # padding is for computing change from one sample to another in the batch

    a = tf.cast(tf.not_equal(sample_ids_in_batch[1:], sample_ids_in_batch[:-1]), tf.int64)
    # for the example above the result of this statement equals
    # [0, 0, 0, 1, 0, 1]
    # so data samples begin in 3rd and 5th positions (the indexes of ones)

    # transforming sample start masks to the sample starts themselves
    q = a * tf.cast(tf.range(n_words), tf.int64)
    # [0, 0, 0, 3, 0, 5]
    count_to_substract = tf.pad(tf.boolean_mask(q, q), [(1, 0)])
    # [0, 3, 5]

    new_word_indices = tf.cast(tf.range(n_words), tf.int64) - tf.gather(count_to_substract, tf.cumsum(a))
    # tf.range(n_words) -> [0, 1, 2, 3, 4, 5]
    # tf.cumsum(a) -> [0, 0, 0, 1, 1, 2]
    # tf.gather(count_to_substract, tf.cumsum(a)) -> [0, 0, 0, 3, 3, 5]
    # new_word_indices -> [0, 1, 2, 3, 4, 5] - [0, 0, 0, 3, 3, 5] = [0, 1, 2, 0, 1, 0]
    # new_word_indices is the concatenation of range(word_len(sentence))
    # for all sentences in units

    n_total_word_elements = tf.cast(batch_size * max_token_seq_len, tf.int32)
    word_indices_flat = tf.cast(idxs[:, 0] * max_token_seq_len + new_word_indices, tf.int32)
    x_mask = tf.reduce_sum(tf.one_hot(word_indices_flat, n_total_word_elements), 0)
    x_mask = tf.cast(x_mask, tf.bool)
    # to get absolute indices we add max_token_seq_len:
    # idxs[:, 0] * max_token_seq_len -> [0, 0, 0, 1, 1, 2] * 2 = [0, 0, 0, 3, 3, 6]
    # word_indices_flat -> [0, 0, 0, 3, 3, 6] + [0, 1, 2, 0, 1, 0] = [0, 1, 2, 3, 4, 6]
    # total number of words in the batch (including paddings)
    # batch_size * max_token_seq_len -> 3 * 3 = 9
    # tf.one_hot(...) ->
    # [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
    #  [0. 1. 0. 0. 0. 0. 0. 0. 0.]
    #  [0. 0. 1. 0. 0. 0. 0. 0. 0.]
    #  [0. 0. 0. 1. 0. 0. 0. 0. 0.]
    #  [0. 0. 0. 0. 1. 0. 0. 0. 0.]
    #  [0. 0. 0. 0. 0. 0. 1. 0. 0.]]
    #  x_mask -> [1, 1, 1, 1, 1, 0, 1, 0, 0]

    full_range = tf.cast(tf.range(batch_size * max_token_seq_len), tf.int32)
    # full_range -> [0, 1, 2, 3, 4, 5, 6, 7, 8]
    nonword_indices_flat = tf.boolean_mask(full_range, tf.math.logical_not(x_mask))
    # # y_idxs -> [5, 7, 8]

    # get a sequence of units corresponding to the start subtokens of the words
    # size: [n_words, n_features]
    elements = tf.gather_nd(units, idxs)

    # prepare zeros for paddings
    # size: [batch_size * TOKEN_seq_length - n_words, n_features]
    paddings = tf.zeros(tf.stack([tf.reduce_sum(max_token_seq_len - token_seq_lengths),
                                  nf], 0), tf.float32)

    tensor_flat = tf.dynamic_stitch([word_indices_flat, nonword_indices_flat],
                                    [elements, paddings])
    # tensor_flat -> [x, x, x, x, x, 0, x, 0, 0]

    tensor = tf.reshape(tensor_flat, tf.stack([batch_size, max_token_seq_len, nf_int], 0))
    # tensor -> [[x, x, x],
    #            [x, x, 0],
    #            [x, 0, 0]]

    return tensor


@register('my_bert_sequence_network')
class MyBertSequenceNetwork(LRScheduledTFModel):
    """
    Basic class for BERT-based sequential architectures.
    Args:

        keep_prob: dropout keep_prob for non-Bert layers
        bert_config_file: path to Bert configuration file
        pretrained_bert: pretrained Bert checkpoint
        attention_probs_keep_prob: keep_prob for Bert self-attention layers
        hidden_keep_prob: keep_prob for Bert hidden layers
        encoder_layer_ids: list of averaged layers from Bert encoder (layer ids)
            optimizer: name of tf.train.* optimizer or None for `AdamWeightDecayOptimizer`
            weight_decay_rate: L2 weight decay for `AdamWeightDecayOptimizer`
        encoder_dropout: dropout probability of encoder output layer
        ema_decay: what exponential moving averaging to use for network parameters, value from 0.0 to 1.0.
            Values closer to 1.0 put weight on the parameters history and values closer to 0.0 corresponds put weight
            on the current parameters.
        ema_variables_on_cpu: whether to put EMA variables to CPU. It may save a lot of GPU memory
        freeze_embeddings: set True to not train input embeddings set True to
            not train input embeddings set True to not train input embeddings
        learning_rate: learning rate of BERT head
        bert_learning_rate: learning rate of BERT body
        min_learning_rate: min value of learning rate if learning rate decay is used
        learning_rate_drop_patience: how many validations with no improvements to wait
        learning_rate_drop_div: the divider of the learning rate after `learning_rate_drop_patience` unsuccessful
            validations
        load_before_drop: whether to load best model before dropping learning rate or not
        clip_norm: clip gradients by norm
    """

    def __init__(self,
                 keep_prob: float,
                 bert_config_file: str,
                 pretrained_bert: str = None,
                 attention_probs_keep_prob: float = None,
                 hidden_keep_prob: float = None,
                 encoder_layer_ids: List[int] = (-1,),
                 encoder_dropout: float = 0.0,
                 optimizer: str = None,
                 weight_decay_rate: float = 1e-6,
                 ema_decay: float = None,
                 ema_variables_on_cpu: bool = True,
                 freeze_embeddings: bool = False,
                 learning_rate: float = 1e-3,
                 bert_learning_rate: float = 2e-5,
                 min_learning_rate: float = 1e-07,
                 learning_rate_drop_patience: int = 20,
                 learning_rate_drop_div: float = 2.0,
                 load_before_drop: bool = True,
                 clip_norm: float = 1.0,
                 **kwargs) -> None:
        super().__init__(learning_rate=learning_rate,
                         learning_rate_drop_div=learning_rate_drop_div,
                         learning_rate_drop_patience=learning_rate_drop_patience,
                         load_before_drop=load_before_drop,
                         clip_norm=clip_norm,
                         **kwargs)
        self.keep_prob = keep_prob
        self.encoder_layer_ids = encoder_layer_ids
        self.encoder_dropout = encoder_dropout
        self.optimizer = optimizer
        self.weight_decay_rate = weight_decay_rate
        self.ema_decay = ema_decay
        self.ema_variables_on_cpu = ema_variables_on_cpu
        self.freeze_embeddings = freeze_embeddings
        self.bert_learning_rate_multiplier = bert_learning_rate / learning_rate
        self.min_learning_rate = min_learning_rate

        self.bert_config = BertConfig.from_json_file(str(expand_path(bert_config_file)))

        if attention_probs_keep_prob is not None:
            self.bert_config.attention_probs_dropout_prob = 1.0 - attention_probs_keep_prob
        if hidden_keep_prob is not None:
            self.bert_config.hidden_dropout_prob = 1.0 - hidden_keep_prob

        self.sess_config = tf.ConfigProto(allow_soft_placement=True)
        self.sess_config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=self.sess_config)

        self._init_graph()

        self._init_optimizer()

        self.sess.run(tf.global_variables_initializer())

        if pretrained_bert is not None:
            pretrained_bert = str(expand_path(pretrained_bert))

            if tf.train.checkpoint_exists(pretrained_bert) \
                    and not (self.load_path and tf.train.checkpoint_exists(str(self.load_path.resolve()))):
                log.info('[initializing model with Bert from {}]'.format(pretrained_bert))
                # Exclude optimizer and classification variables from saved variables
                var_list = self._get_saveable_variables(
                    exclude_scopes=('Optimizer', 'learning_rate', 'momentum', 'ner', 'EMA'))
                saver = tf.train.Saver(var_list)
                saver.restore(self.sess, pretrained_bert)

        if self.load_path is not None:
            self.load()

        if self.ema:
            self.sess.run(self.ema.init_op)

    def _init_graph(self) -> None:
        self.seq_lengths = tf.reduce_sum(self.y_masks_ph, axis=1)

        self.bert = BertModel(config=self.bert_config,
                              is_training=self.is_train_ph,
                              input_ids=self.input_ids_ph,
                              input_mask=self.input_masks_ph,
                              token_type_ids=self.token_types_ph,
                              use_one_hot_embeddings=False)
        with tf.variable_scope('ner'):
            layer_weights = tf.get_variable('layer_weights_',
                                            shape=len(self.encoder_layer_ids),
                                            initializer=tf.ones_initializer(),
                                            trainable=True)
            layer_mask = tf.ones_like(layer_weights)
            layer_mask = tf.nn.dropout(layer_mask, self.encoder_keep_prob_ph)
            layer_weights *= layer_mask
            # to prevent zero division
            mask_sum = tf.maximum(tf.reduce_sum(layer_mask), 1.0)
            layer_weights = tf.unstack(layer_weights / mask_sum)
            # TODO: may be stack and reduce_sum is faster
            units = sum(w * l for w, l in zip(layer_weights, self.encoder_layers()))
            units = tf.nn.dropout(units, keep_prob=self.keep_prob_ph)
            # print('init graph var scope ner')
        return units

    def _get_tag_mask(self) -> tf.Tensor:
        """
        Returns: tag_mask,
            a mask that selects positions corresponding to word tokens (not padding and `CLS`)
        """
        max_length = tf.reduce_max(self.seq_lengths)
        one_hot_max_len = tf.one_hot(self.seq_lengths - 1, max_length)
        tag_mask = tf.cumsum(one_hot_max_len[:, ::-1], axis=1)[:, ::-1]

        return tag_mask

    def encoder_layers(self):
        """
        Returns: the output of BERT layers specfied in ``self.encoder_layers_ids``
        """
        return [self.bert.all_encoder_layers[i] for i in self.encoder_layer_ids]

    def _init_placeholders(self) -> None:
        self.input_ids_ph = tf.placeholder(shape=(None, None),
                                           dtype=tf.int32,
                                           name='token_indices_ph')
        self.input_masks_ph = tf.placeholder(shape=(None, None),
                                             dtype=tf.int32,
                                             name='token_mask_ph')
        self.token_types_ph = \
            tf.placeholder_with_default(tf.zeros_like(self.input_ids_ph, dtype=tf.int32),
                                        shape=self.input_ids_ph.shape,
                                        name='token_types_ph')
        self.learning_rate_ph = tf.placeholder_with_default(0.0, shape=[], name='learning_rate_ph')
        self.keep_prob_ph = tf.placeholder_with_default(1.0, shape=[], name='keep_prob_ph')
        self.encoder_keep_prob_ph = tf.placeholder_with_default(1.0, shape=[], name='encoder_keep_prob_ph')
        self.is_train_ph = tf.placeholder_with_default(False, shape=[], name='is_train_ph')

    def _init_optimizer(self) -> None:
        with tf.variable_scope('Optimizer'):
            self.global_step = tf.get_variable('global_step',
                                               shape=[],
                                               dtype=tf.int32,
                                               initializer=tf.constant_initializer(0),
                                               trainable=False)
            # default optimizer for Bert is Adam with fixed L2 regularization

        if self.optimizer is None:
            self.train_op = \
                self.get_train_op(self.loss,
                                  learning_rate=self.learning_rate_ph,
                                  optimizer=AdamWeightDecayOptimizer,
                                  weight_decay_rate=self.weight_decay_rate,
                                  beta_1=0.9,
                                  beta_2=0.999,
                                  epsilon=1e-6,
                                  optimizer_scope_name='Optimizer',
                                  exclude_from_weight_decay=["LayerNorm",
                                                             "layer_norm",
                                                             "bias",
                                                             "EMA"])
        else:
            self.train_op = self.get_train_op(self.loss,
                                              learning_rate=self.learning_rate_ph,
                                              optimizer_scope_name='Optimizer')

        if self.optimizer is None:
            with tf.variable_scope('Optimizer'):
                new_global_step = self.global_step + 1
                self.train_op = tf.group(self.train_op, [self.global_step.assign(new_global_step)])

        if self.ema_decay is not None:
            _vars = self._get_trainable_variables(exclude_scopes=["Optimizer",
                                                                  "LayerNorm",
                                                                  "layer_norm",
                                                                  "bias",
                                                                  "learning_rate",
                                                                  "momentum"])

            self.ema = ExponentialMovingAverage(self.ema_decay,
                                                variables_on_cpu=self.ema_variables_on_cpu)
            self.train_op = self.ema.build(self.train_op, _vars, name="EMA")
        else:
            self.ema = None

    def get_train_op(self, loss: tf.Tensor, learning_rate: Union[tf.Tensor, float], **kwargs) -> tf.Operation:
        assert "learnable_scopes" not in kwargs, "learnable scopes unsupported"
        # train_op for bert variables
        kwargs['learnable_scopes'] = ('bert/encoder', 'bert/embeddings')
        if self.freeze_embeddings:
            kwargs['learnable_scopes'] = ('bert/encoder',)
        bert_learning_rate = learning_rate * self.bert_learning_rate_multiplier
        bert_train_op = super().get_train_op(loss,
                                             bert_learning_rate,
                                             **kwargs)
        # train_op for ner head variables
        kwargs['learnable_scopes'] = ('ner',)
        # print('1 get train op')
        head_train_op = super().get_train_op(loss,
                                             learning_rate,
                                             **kwargs)
        return tf.group(bert_train_op, head_train_op)

    def _build_basic_feed_dict(self, input_ids: tf.Tensor, input_masks: tf.Tensor,
                               token_types: Optional[tf.Tensor]=None, train: bool=False) -> dict:
        """Fills the feed_dict with the tensors defined in the basic class.
        You need to update this dict by the values of output placeholders
        and class-specific network inputs in your derived class.
        """
        # print('1 _build_basic_feed_dict')
        feed_dict = {
            self.input_ids_ph: input_ids,
            self.input_masks_ph: input_masks,
        }
        if token_types is not None:
            feed_dict[self.token_types_ph] = token_types
        if train:
            # print('1 _build_basic_feed_dict train')
            feed_dict.update({
                self.learning_rate_ph: max(self.get_learning_rate(), self.min_learning_rate),
                self.keep_prob_ph: self.keep_prob,
                self.encoder_keep_prob_ph: 1.0 - self.encoder_dropout,
                self.is_train_ph: True,
            })

        return feed_dict

    def _build_feed_dict(self, input_ids, input_masks, token_types=None, *args,  **kwargs):
        raise NotImplementedError("You must implement _build_feed_dict in your derived class.")

    def train_on_batch(self,
                       input_ids: Union[List[List[int]], np.ndarray],
                       input_masks: Union[List[List[int]], np.ndarray],
                      #  topk_tok_mask: Union[List[List[int]], np.ndarray]
                       *args, **kwargs) -> Dict[str, float]:
        """
        Args:
            input_ids: batch of indices of subwords
            input_masks: batch of masks which determine what should be attended
            args: arguments passed  to _build_feed_dict
                and corresponding to additional input
                and output tensors of the derived class.
            kwargs: keyword arguments passed to _build_feed_dict
                and corresponding to additional input
                and output tensors of the derived class.

        Returns:
            dict with fields 'loss', 'head_learning_rate', and 'bert_learning_rate'
        """
        # print('1 tr_on_batch1')
        feed_dict = self._build_feed_dict(input_ids, input_masks, *args, **kwargs)
        # print('1 feed_dict_tr_batch')
        # print('1 tr_on_batch2')
        if self.ema:
            self.sess.run(self.ema.switch_to_train_op)
        # print('1 sess with loss')
        # print(self.topk_tok_mask)
        _, loss, lr = self.sess.run([self.train_op, self.loss, self.learning_rate_ph],
                                     feed_dict=feed_dict)

        return {'loss': loss,
                'head_learning_rate': float(lr),
                'bert_learning_rate': float(lr) * self.bert_learning_rate_multiplier}

    def __call__(self,
                 input_ids: Union[List[List[int]], np.ndarray],
                 input_masks: Union[List[List[int]], np.ndarray],
                 **kwargs) -> Union[List[List[int]], List[np.ndarray]]:
        raise NotImplementedError("You must implement method __call__ in your derived class.")

    def save(self, exclude_scopes=('Optimizer', 'EMA/BackupVariables')) -> None:
        if self.ema:
            self.sess.run(self.ema.switch_to_train_op)
        return super().save(exclude_scopes=exclude_scopes)

    def load(self,
             exclude_scopes=('Optimizer',
                             'learning_rate',
                             'momentum',
                             'EMA/BackupVariables'),
             **kwargs) -> None:
        return super().load(exclude_scopes=exclude_scopes, **kwargs)


@register('my_bert_sequence_tagger')
class MyBertSequenceTagger(MyBertSequenceNetwork):
    """BERT-based model for text tagging. It predicts a label for every token (not subtoken) in the text.
    You can use it for sequence labeling tasks, such as morphological tagging or named entity recognition.
    See :class:`deeppavlov.models.bert.bert_sequence_tagger.BertSequenceNetwork`
    for the description of inherited parameters.
    Args:
        n_tags: number of distinct tags
        use_crf: whether to use CRF on top or not
        use_birnn: whether to use bidirection rnn after BERT layers.
            For NER and morphological tagging we usually set it to `False` as otherwise the model overfits
        birnn_cell_type: the type of Bidirectional RNN. Either `lstm` or `gru`
        birnn_hidden_size: number of hidden units in the BiRNN layer in each direction
        return_probas: set this to `True` if you need the probabilities instead of raw answers
    """

    def __init__(self,
                 n_tags: List[str],
                 keep_prob: float,
                 bert_config_file: str,
                 pretrained_bert: str = None,
                 attention_probs_keep_prob: float = None,
                 hidden_keep_prob: float = None,
                 use_crf=False,
                 encoder_layer_ids: List[int] = (-1,),
                 encoder_dropout: float = 0.0,
                 optimizer: str = None,
                 weight_decay_rate: float = 1e-6,
                 use_birnn: bool = False,
                 birnn_cell_type: str = 'lstm',
                 birnn_hidden_size: int = 128,
                 ema_decay: float = None,
                 ema_variables_on_cpu: bool = True,
                 return_probas: bool = False,
                 freeze_embeddings: bool = False,
                 learning_rate: float = 1e-3,
                 bert_learning_rate: float = 2e-5,
                 min_learning_rate: float = 1e-07,
                 learning_rate_drop_patience: int = 20,
                 learning_rate_drop_div: float = 2.0,
                 load_before_drop: bool = True,
                 clip_norm: float = 1.0,
                 **kwargs) -> None:
        self.n_tags = n_tags
        self.use_crf = use_crf
        self.use_birnn = use_birnn
        self.birnn_cell_type = birnn_cell_type
        self.birnn_hidden_size = birnn_hidden_size
        self.return_probas = return_probas
        super().__init__(keep_prob=keep_prob,
                         bert_config_file=bert_config_file,
                         pretrained_bert=pretrained_bert,
                         attention_probs_keep_prob=attention_probs_keep_prob,
                         hidden_keep_prob=hidden_keep_prob,
                         encoder_layer_ids=encoder_layer_ids,
                         encoder_dropout=encoder_dropout,
                         optimizer=optimizer,
                         weight_decay_rate=weight_decay_rate,
                         ema_decay=ema_decay,
                         ema_variables_on_cpu=ema_variables_on_cpu,
                         freeze_embeddings=freeze_embeddings,
                         learning_rate=learning_rate,
                         bert_learning_rate=bert_learning_rate,
                         min_learning_rate=min_learning_rate,
                         learning_rate_drop_div=learning_rate_drop_div,
                         learning_rate_drop_patience=learning_rate_drop_patience,
                         load_before_drop=load_before_drop,
                         clip_norm=clip_norm,
                         **kwargs)

    def _init_graph(self) -> None:
        self._init_placeholders()

        units = super()._init_graph()

        with tf.variable_scope('ner'):
            if self.use_birnn:
                units, _ = bi_rnn(units,
                                  self.birnn_hidden_size,
                                  cell_type=self.birnn_cell_type,
                                  seq_lengths=self.seq_lengths,
                                  name='birnn')
                units = tf.concat(units, -1)
            # TODO: maybe add one more layer?
            logits = tf.layers.dense(units, units=self.n_tags, name="output_dense")
            self.logits = token_from_subtoken(logits, self.y_masks_ph) #token_ids of startwords?

            # CRF
            if self.use_crf:
                transition_params = tf.get_variable('Transition_Params',
                                                    shape=[self.n_tags, self.n_tags],
                                                    initializer=tf.zeros_initializer())
                log_likelihood, transition_params = \
                    tf.contrib.crf.crf_log_likelihood(self.logits,
                                                      self.y_ph,
                                                      self.seq_lengths,
                                                      transition_params)
                loss_tensor = -log_likelihood
                self._transition_params = transition_params

            self.y_predictions = tf.argmax(self.logits, -1)
            self.y_probas = tf.nn.softmax(self.logits, axis=2)

        with tf.variable_scope("loss"):
            tag_mask = self._get_tag_mask()
            y_mask = tf.cast(tag_mask, tf.float32)
            if self.use_crf:
                self.loss = tf.reduce_mean(loss_tensor)
            else:                
                self.loss = tf.losses.sparse_softmax_cross_entropy(labels=self.y_ph,
                                                                   logits=self.logits,
                                                                   weights=self.topk_masks_ph)  #instead of y_mask


    def _init_placeholders(self) -> None:
        super()._init_placeholders()
        self.y_ph = tf.placeholder(shape=(None, None), dtype=tf.int32, name='y_ph')
        self.y_masks_ph = tf.placeholder(shape=(None, None),
                                         dtype=tf.int32,
                                         name='y_mask_ph')
        self.topk_masks_ph = tf.placeholder(shape=(None, None),  # added
                                         dtype=tf.int32,
                                         name='topk_mask_ph')

    def _decode_crf(self, feed_dict: Dict[tf.Tensor, np.ndarray]) -> List[np.ndarray]:
        logits, trans_params, mask, seq_lengths = self.sess.run([self.logits,
                                                                 self._transition_params,
                                                                 self.y_masks_ph,
                                                                 self.seq_lengths],
                                                                feed_dict=feed_dict)
        # iterate over the sentences because no batching in viterbi_decode
        y_pred = []
        for logit, sequence_length in zip(logits, seq_lengths):
            logit = logit[:int(sequence_length)]  # keep only the valid steps
            viterbi_seq, viterbi_score = tf.contrib.crf.viterbi_decode(logit, trans_params)
            y_pred += [viterbi_seq]
        return y_pred

    def _build_feed_dict(self, input_ids, input_masks, y_masks, topk_tok_mask, y=None):
        feed_dict = self._build_basic_feed_dict(input_ids, input_masks, train=(y is not None))

        if y is not None:
            max_len = 0
            for seq in topk_tok_mask:
                if len(seq) > max_len:
                    max_len = len(seq)
            feed_dict[self.topk_masks_ph] = topk_tok_mask # added            
            feed_dict[self.y_ph] = y
        feed_dict[self.y_masks_ph] = y_masks

        return feed_dict

    def __call__(self,
                 input_ids: Union[List[List[int]], np.ndarray],
                 input_masks: Union[List[List[int]], np.ndarray],
                 y_masks: Union[List[List[int]], np.ndarray],
                 topk_tok_mask: Union[List[List[int]], np.ndarray]) -> Union[List[List[int]], List[np.ndarray]]:
        """ Predicts tag indices for a given subword tokens batch
        Args:
            input_ids: indices of the subwords
            input_masks: mask that determines where to attend and where not to
            y_masks: mask which determines the first subword units in the the word

        Added: 
            topk_tok_mask: mask that determines where to attend and where not to
        Returns:
            Label indices or class probabilities for each token (not subtoken)
        # """
        
        feed_dict = self._build_feed_dict(input_ids, input_masks, y_masks, topk_tok_mask)
       
        if self.ema:
            self.sess.run(self.ema.switch_to_test_op)
        if not self.return_probas:
            if self.use_crf:
                pred = self._decode_crf(feed_dict)
            else:
                pred, seq_lengths = self.sess.run([self.y_predictions, self.seq_lengths], feed_dict=feed_dict)
                pred = [p[:l] for l, p in zip(seq_lengths, pred)]
        else:
            pred = self.sess.run(self.y_probas, feed_dict=feed_dict)

        return pred


class ExponentialMovingAverage:
    def __init__(self,
                 decay: float = 0.999,
                 variables_on_cpu: bool = True) -> None:
        self.decay = decay
        self.ema = tf.train.ExponentialMovingAverage(decay=decay)
        self.var_device_name = '/cpu:0' if variables_on_cpu else None
        self.train_mode = None

    def build(self,
              minimize_op: tf.Tensor,
              update_vars: List[tf.Variable] = None,
              name: str = "EMA") -> tf.Tensor:
        with tf.variable_scope(name):
            if update_vars is None:
                update_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

            with tf.control_dependencies([minimize_op]):
                minimize_op = self.ema.apply(update_vars)

            with tf.device(self.var_device_name):
                # Make backup variables
                with tf.variable_scope('BackupVariables'):
                    backup_vars = [tf.get_variable(var.op.name,
                                                   dtype=var.value().dtype,
                                                   trainable=False,
                                                   initializer=var.initialized_value())
                                   for var in update_vars]

                def ema_to_weights():
                    return tf.group(*(tf.assign(var, self.ema.average(var).read_value())
                                      for var in update_vars))

                def save_weight_backups():
                    return tf.group(*(tf.assign(bck, var.read_value())
                                      for var, bck in zip(update_vars, backup_vars)))

                def restore_weight_backups():
                    return tf.group(*(tf.assign(var, bck.read_value())
                                      for var, bck in zip(update_vars, backup_vars)))

                train_switch_op = restore_weight_backups()
                with tf.control_dependencies([save_weight_backups()]):
                    test_switch_op = ema_to_weights()

            self.train_switch_op = train_switch_op
            self.test_switch_op = test_switch_op
            self.do_nothing_op = tf.no_op()

        return minimize_op

    @property
    def init_op(self) -> tf.Operation:
        self.train_mode = False
        return self.test_switch_op

    @property
    def switch_to_train_op(self) -> tf.Operation:
        assert self.train_mode is not None, "ema variables aren't initialized"
        if not self.train_mode:
            # log.info("switching to train mode")
            self.train_mode = True
            return self.train_switch_op
        return self.do_nothing_op

    @property
    def switch_to_test_op(self) -> tf.Operation:
        assert self.train_mode is not None, "ema variables aren't initialized"
        if self.train_mode:
            # log.info("switching to test mode")
            self.train_mode = False
            return self.test_switch_op
        return self.do_nothing_op

## Return Error

Component that masks tokens for the evaluation, so the metrics are counted considering certain group of tokens. By masking here I mean just forming new lists of tokens without the irrelevant ones.

Also it forms 3 files in the models folder:
- correct.txt - all the correctly tagged tokens, token/predicted/sentence
- error.txt - all the wrongly tagged tokens, token/predicted/true/sentence
- count.txt - number of tokens, to compute accuracy later, overall_tokens/correct_tokens
- preds.txt - predictions of the classifier

Not elegant at all, but was a little limited by deeppavlov's functions arrangement

In [ ]:
@register('Info_logger')
class Info_logger(Component):
    def __init__(self, info_path, **kwargs):
        os.makedirs(info_path, exist_ok=True)

        self.error_path = info_path + '/errors.txt'
        self.correct_path = info_path + '/correct.txt'
        self.count_path = info_path + '/count.txt'
        self.preds_path = info_path + '/preds.txt'


    def __call__(self, y, topk_tok_mask, y_predicted, x_words, *args, **kwargs):

        error_file = open(self.error_path, 'a')  # token/predicted/true/sentence
        correct_file = open(self.correct_path, 'a') # token/predicted/sentence
        count_file = open(self.count_path, 'a') # overall_tokens/correct_tokens
        preds_file = open(self.preds_path, 'a') #

        y_predicted_masked = []
        y_masked = []
        overall_tokens = 0
        correct_tokens = 0

        for i in range(len(y_predicted)):
            line_preds = []
            line_true = []
            if x_words[i] != []:
                sentence = ' '.join(x_words[i])
            for j in range(len(y_predicted[i])):
                if topk_tok_mask[i][j] == 1:
                    overall_tokens += 1
                    line_preds.append(y_predicted[i][j])
                    line_true.append(y[i][j])
                    preds_file.write(x_words[i][j] + '\t' + y_predicted[i][j] + '\t' + sentence + '\n')
                    if y_predicted[i][j] == y[i][j]:
                        correct_tokens += 1
                        correct_file.write(x_words[i][j] + '\t' + y[i][j] + '\t' + sentence + '\n')
                    else:
                        error_file.write(x_words[i][j] + '\t' + y_predicted[i][j] + '\t' + y[i][j] + '\t' + sentence + '\n')

            y_predicted_masked.append(np.array(line_preds))
            y_masked.append(np.array(line_true))

        
        count_file.write(str(overall_tokens)+'\t'+str(correct_tokens)+'\n')

        error_file.close()
        correct_file.close()
        count_file.close()
        # ic (y_predicted_masked)
        # ic (y_masked)
        return y_masked, y_predicted_masked
        

# Evaluation

need to specify in config:
- ['dataset_reader']['data_path'] - path to datafolder
- ['metadata']['variables']['MODELS_PATH'] - path to general folder concerning to certain dataset
- ['train']['evaluation_targets'] = valid, ['dataset_reader']['data_types'] = ['train', 'dev'], if there is no test file
- ['metadata']['variables']['DATA_PATH'] - path to datafolder

will be specified below:
- ['metadata']['variables']['WORK_PATH'] - path to certain model folder
- ['chainer']['pipe'][1]['topk_tokens_path'] - path to the group of tokens to consider

In [ ]:
#START 
# alter config4eval from the newest base_config 

In [ ]:
import pandas as pd

import json
from deeppavlov import build_model, configs, evaluate_model

In [ ]:
config_eval = json.load(open('./config_eval.json'))

config_eval['metadata']['variables']['DATA_PATH'] = datafolder_path
config_eval['metadata']['variables']['MODELS_PATH'] = '{ROOT_PATH}/drive/MyDrive/models_diploma/GramEval_social'
# config_eval['train']['evaluation_targets'] = ['valid']
# config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['dataset_reader']['language'] = language_conf


In [ ]:
def count_tokens(f):

    overall_tokens, correct_tokens = 0, 0

    for line in f.readlines():
        overall_tokens += int(line.split('\t')[0])
        
        correct_tokens += int(line.split('\t')[1])

    return overall_tokens, correct_tokens

def full_report(path, data_type='valid'): #path to models folder

    top100_f = open(path + '/model_top100/info/' + data_type + '/count.txt', 'r')
    top1000_f = open(path + '/model_top1000/info/' + data_type + '/count.txt', 'r')
    other_f = open(path + '/model_other/info/' + data_type + '/count.txt', 'r')

    rez = {'model': ['top100', 'top1000', 'other', 'general'], 
           'accuracy': [],
            }

    total, correct = 0, 0
    for f in [top100_f, top1000_f, other_f]:
         overall_tokens, correct_tokens = count_tokens(f)
         rez['accuracy'].append(round(correct_tokens/overall_tokens, 4))
         total += overall_tokens
         correct += correct_tokens
    
    rez['accuracy'].append(round(correct/total, 4))
    
    rez_df = pd.DataFrame(rez)

    top100_f.close()
    top1000_f.close()
    other_f.close()

    return rez_df



## From Scratch Evaluation

###top100

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/fs_models/model_top100'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt']
config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']

config_eval['chainer']['pipe'][1]['last'] = False
config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:09:24.930 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:


2022-05-26 17:09:26.367 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top100/tag.dict]






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2022-05-26 17:09:44.474 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top100/model]



INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top100/model


2022-05-26 17:09:54.196 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.9046, "accuracy": 0.7727}, "time_spent": "0:00:02"}}


###top1000

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/fs_models/model_top1000'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/1.txt']

config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['chainer']['pipe'][1]['last'] = False

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:09:56.357 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:09:57.424 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top1000/tag.dict]
2022-05-26 17:10:12.470 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top1000/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_top1000/model


2022-05-26 17:10:18.706 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.8397, "accuracy": 0.7614}, "time_spent": "0:00:02"}}


###other

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/fs_models/model_other'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt', '{MODELS_PATH}/freq_groups/1.txt']
config_eval['chainer']['pipe'][1]['last'] = True


config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:10:20.81 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:10:20.760 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_other/tag.dict]
2022-05-26 17:10:36.727 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_other/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/fs_models/model_other/model


2022-05-26 17:10:44.75 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.919, "accuracy": 0.6477}, "time_spent": "0:00:02"}}


### Report

In [ ]:
path = '/content/drive/MyDrive/models_diploma/GramEval_social/fs_models' #maybe wrong
rez_val = full_report(path, data_type='valid')

In [ ]:
rez_val

,model,accuracy
0,top100,0.9046
1,top1000,0.8397
2,other,0.9190
3,general,0.9027


## Baseline Evaluation 

### Baseline on 100

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/baseline'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt']
config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['chainer']['pipe'][1]['last'] = False

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/model_top100/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)


2022-05-26 17:10:45.698 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:10:46.170 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/tag.dict]
2022-05-26 17:11:02.107 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model


2022-05-26 17:11:07.531 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.9112, "accuracy": 0.7955}, "time_spent": "0:00:02"}}


### Baseline 1000

In [ ]:
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/1.txt']
config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/model_top1000'
config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['chainer']['pipe'][1]['last'] = False

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/model_top1000/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:11:08.975 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:11:09.330 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/tag.dict]
2022-05-26 17:11:24.492 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model


2022-05-26 17:11:26.329 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.8269, "accuracy": 0.75}, "time_spent": "0:00:02"}}


### Baseline 1000+

In [ ]:
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt', '{MODELS_PATH}/freq_groups/1.txt']
config_eval['chainer']['pipe'][1]['last'] = True
config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']


config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/model_other/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:11:27.759 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:11:28.120 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/tag.dict]
2022-05-26 17:11:44.785 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model


2022-05-26 17:11:46.565 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.919, "accuracy": 0.6477}, "time_spent": "0:00:02"}}


### Report

In [ ]:
path = '/content/drive/MyDrive/models_diploma/GramEval_social/baseline' #maybe wrong
rez = full_report(path, data_type='valid')

In [ ]:
rez

,model,accuracy
0,top100,0.9112
1,top1000,0.8269
2,other,0.9190
3,general,0.9027


## FineTuned Model Evaluation

###top100

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/finetuned_models/model_top100'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt']
config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['chainer']['pipe'][1]['last'] = False

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:11:48.18 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:11:48.756 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top100/tag.dict]
2022-05-26 17:12:05.454 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top100/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top100/model


2022-05-26 17:12:12.471 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.9211, "accuracy": 0.7955}, "time_spent": "0:00:02"}}


###top1000

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/finetuned_models/model_top1000'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/1.txt']

config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']
config_eval['chainer']['pipe'][1]['last'] = False

config_eval['chainer']['pipe'][7]['info_path'] = '{WORK_PATH}/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:12:13.967 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:12:14.770 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top1000/tag.dict]
2022-05-26 17:12:31.624 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top1000/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models/model_top1000/model


2022-05-26 17:12:43.21 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.859, "accuracy": 0.7727}, "time_spent": "0:00:02"}}


###other = baseline as it did not improve

In [ ]:
config_eval['metadata']['variables']['WORK_PATH'] = '{MODELS_PATH}/baseline'
config_eval['chainer']['pipe'][1]['topk_tokens_path'] = ['{MODELS_PATH}/freq_groups/0.txt', '{MODELS_PATH}/freq_groups/1.txt']
config_eval['chainer']['pipe'][1]['last'] = True


config_eval['train']['evaluation_targets'] = ['valid']
config_eval['dataset_reader']['data_types'] = ['train', 'dev']

config_eval['chainer']['pipe'][7]['info_path'] = '{MODELS_PATH}/finetuned_models/model_other/info/valid'

In [ ]:
model = evaluate_model(config_eval, download=False)

2022-05-26 17:12:44.511 INFO in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 68: NNTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
2022-05-26 17:12:44.874 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/tag.dict]
2022-05-26 17:13:01.753 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model]


INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/models_diploma/GramEval_social/baseline/model


2022-05-26 17:13:03.618 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!


{"valid": {"eval_examples_count": 88, "metrics": {"per_token_accuracy": 0.919, "accuracy": 0.6477}, "time_spent": "0:00:02"}}


### Report

In [ ]:
path = '/content/drive/MyDrive/models_diploma/GramEval_social/finetuned_models' #maybe wrong
rez_val = full_report(path, data_type='valid')

In [ ]:
rez_val

,model,accuracy
0,top100,0.9211
1,top1000,0.8590
2,other,0.9190
3,general,0.9105
